In [2]:
!docker-compose up -d db

!docker-compose ps

[+] Running 1/2
 ✔ Network imdb-project_default  Created                                   0.0s 
 ⠋ Container imdb-db             Starting                                  0.1s 
[+] Running 2/2
 ✔ Network imdb-project_default  Created                                   0.0s 
 ✔ Container imdb-db             Started                                   0.1s 
NAME      IMAGE         COMMAND                  SERVICE   CREATED                  STATUS                                     PORTS
imdb-db   postgres:18   "docker-entrypoint.s…"   db        Less than a second ago   Up Less than a second (health: starting)   0.0.0.0:5432->5432/tcp, [::]:5432->5432/tcp


In [12]:
import pandas as pd
import gzip
import requests
import shutil
import os
from sys import path
from sqlalchemy import create_engine
from dotenv import load_dotenv
import logging

logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')

path.append(os.path.realpath("../"))
load_dotenv()

NAME_BASIC_URL = "https://datasets.imdbws.com/name.basics.tsv.gz"

NAMES_ZIP_FILE = "name.basics.tsv.gz"
NAMES_FILE = "name.basics.tsv"

if not os.path.isfile(NAMES_FILE):
    response = requests.get(NAME_BASIC_URL, stream=True)
    with open(NAMES_ZIP_FILE, "wb") as f:
        shutil.copyfileobj(response.raw, f)

    with gzip.open("name.basics.tsv.gz", "rb") as f_in:
        with open("name.basics.tsv", "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)

df_actors = pd.read_csv("name.basics.tsv", sep="\t")


selected_columns = [
    "nconst",
    "primaryName",
    "birthYear",
    "primaryProfession"
]
clean_df = df_actors[selected_columns]
engine = create_engine(os.getenv("DATABASE_URL"))
try:
    table_name = "actors"
    clean_df.to_sql(table_name,engine, if_exists="replace", index=False)
    table = pd.read_sql("SELECT * FROM actors LIMIT 10", engine)
    logging.info(f"\n{table}") 
except Exception as e:
    logging.error("Exception:", e)
finally:
    engine.dispose()

2025-12-06 22:18:37,152 - INFO - 
      nconst      primaryName birthYear                  primaryProfession
0  nm0000001     Fred Astaire      1899       actor,miscellaneous,producer
1  nm0000002    Lauren Bacall      1924   actress,miscellaneous,soundtrack
2  nm0000003  Brigitte Bardot      1934  actress,music_department,producer
3  nm0000004     John Belushi      1949      actor,writer,music_department
4  nm0000005   Ingmar Bergman      1918              writer,director,actor
5  nm0000006   Ingrid Bergman      1915        actress,producer,soundtrack
6  nm0000007  Humphrey Bogart      1899       actor,producer,miscellaneous
7  nm0000008    Marlon Brando      1924              actor,director,writer
8  nm0000009   Richard Burton      1925            actor,producer,director
9  nm0000010     James Cagney      1899            actor,director,producer


In [ ]:
import logging
import os
import sys
import gzip
import shutil
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from io import StringIO
import csv

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

sys.path.append(os.path.realpath("../"))
load_dotenv()

NAME_BASIC_URL = "https://datasets.imdbws.com/name.basics.tsv.gz"
NAMES_ZIP_FILE = "name.basics.tsv.gz"
NAMES_FILE = "name.basics.tsv"

if not os.path.isfile(NAMES_FILE):
    logging.info("Downloading name.basics.tsv.gz...")
    response = requests.get(NAME_BASIC_URL, stream=True)
    with open(NAMES_ZIP_FILE, "wb") as f:
        shutil.copyfileobj(response.raw, f)
    
    logging.info("Extracting gzip file...")
    with gzip.open(NAMES_ZIP_FILE, "rb") as f_in:
        with open(NAMES_FILE, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    logging.info("Download complete")

def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join(['"{}"'.format(k) for k in keys])
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(os.getenv("DATABASE_URL"))
table_name = "actors"

try:
    logging.info("Reading TSV file in chunks...")
    selected_columns = ["nconst", "primaryName", "birthYear", "primaryProfession"]
    chunk_size = 100000
    
    for i, chunk in enumerate(pd.read_csv(
        NAMES_FILE, 
        sep="\t", 
        usecols=selected_columns,
        chunksize=chunk_size,
        na_values=['\\N'],
        keep_default_na=True,
        dtype={
        'nconst': str,
        'primaryName': str,
        'birthYear': 'Int64',
        'primaryProfession': str
    }
    )):
        chunk = chunk.where(pd.notnull(chunk), None)

        if_exists_mode = 'replace' if i == 0 else 'append'
        
        chunk.to_sql(
            table_name,
            engine,
            if_exists=if_exists_mode,
            index=False,
            method=psql_insert_copy
        )
        
        logging.info(f"Inserted chunk {i+1} ({len(chunk)} rows)")
    
    with engine.connect() as conn:
        result = pd.read_sql("SELECT COUNT(*) as total FROM actors", conn)
        logging.info(f"Total rows inserted: {result['total'][0]}")
    
except Exception as e:
    logging.error(f"Exception occurred: {e}")
    raise
finally:
    engine.dispose()
    logging.info("Database connection closed")

2025-12-06 23:31:21,927 - INFO - Reading TSV file in chunks...
2025-12-06 23:31:22,195 - INFO - Inserted chunk 1 (100000 rows)
2025-12-06 23:31:22,359 - INFO - Inserted chunk 2 (100000 rows)
2025-12-06 23:31:22,527 - INFO - Inserted chunk 3 (100000 rows)
2025-12-06 23:31:22,693 - INFO - Inserted chunk 4 (100000 rows)
2025-12-06 23:31:22,858 - INFO - Inserted chunk 5 (100000 rows)
2025-12-06 23:31:23,027 - INFO - Inserted chunk 6 (100000 rows)
2025-12-06 23:31:23,194 - INFO - Inserted chunk 7 (100000 rows)
2025-12-06 23:31:23,365 - INFO - Inserted chunk 8 (100000 rows)
2025-12-06 23:31:23,534 - INFO - Inserted chunk 9 (100000 rows)
2025-12-06 23:31:23,686 - INFO - Inserted chunk 10 (100000 rows)
2025-12-06 23:31:23,838 - INFO - Inserted chunk 11 (100000 rows)
2025-12-06 23:31:23,987 - INFO - Inserted chunk 12 (100000 rows)
2025-12-06 23:31:24,138 - INFO - Inserted chunk 13 (100000 rows)
2025-12-06 23:31:24,286 - INFO - Inserted chunk 14 (100000 rows)
2025-12-06 23:31:24,437 - INFO - Ins

## Data insertion

After reviewing several options to insert the data into the postgressql database the best approach would be the copy one.

In [34]:
!docker-compose down -v

[+] Running 0/1
 ⠋ Container imdb-db  Stopping                                             0.1s 
[+] Running 0/1
 ⠙ Container imdb-db  Stopping                                             0.2s 
[+] Running 0/1
 ⠹ Container imdb-db  Stopping                                             0.3s 
[+] Running 1/3
 ✔ Container imdb-db                  Removed                              0.3s 
 ⠋ Volume imdb-project_postgres_data  Removing                             0.1s 
 ⠋ Network imdb-project_default       Re...                                0.1s 
[+] Running 2/3
 ✔ Container imdb-db                  Removed                              0.3s 
 ✔ Volume imdb-project_postgres_data  Removed                              0.1s 
 ⠙ Network imdb-project_default       Re...                                0.2s 
[+] Running 2/3
 ✔ Container imdb-db                  Removed                              0.3s 
 ✔ Volume imdb-project_postgres_data  Removed                              0.1s 
 ⠹ Network im